In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, GRU
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#set(stopwords.words('english'))
from nltk import word_tokenize
#STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#ONLY RUN ON COLAB AND IMPORT DATA FILE FROM LOCAL SYSTEM (cause using drive is a major hassle)
from google.colab import files
uploaded = files.upload()

Saving conversational_data.csv to conversational_data.csv


In [ ]:
#change the name of the file here accordingly
df = pd.read_csv("conversational_data.csv", usecols=['text', 'replies__text'])
df.text=df.text.astype(str)
df.replies__text=df.replies__text.astype(str)
#df=ds.sample(frac=1)
column=[]
df.head()
df.info()
#df = [str(i) for i in df]
for text in df.text:
    print(text)



,text,replies__text
0,So you don't agree that the subgroup from Stu...,"1 2 ""I predict that Gardasil [vaccine] will b..."
1,The sike is for my feeling being hurt ! My dau...,You must really ♥️ her when Immunization prev...
2,Monkey this is a completely pointless tweet. ...,"Hey Mikey, 11 women died yesterday from cervi..."
3,Sanders accepted an endorsement from Wayne La...,Sanders has always placed class above both ge...
4,the weirdest thing about gender reveal parties...,We didn't have 'baby showers' in the UK until ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 2 columns):
text             370 non-null object
replies__text    370 non-null object
dtypes: object(2)
memory usage: 5.9+ KB
 So you don't agree that the subgroup from Study 013 was unbalanced, and that study 015 is more representative (p.14)? Why not?
The sike is for my feeling being hurt ! My daughter 9 months no vaccinations & just left the doctor and is healthy as can be 🙏🏽
 Monkey this is a completely pointless tweet. Stick to just marveling. You seem smarter when you don't engage. 😉
 Sanders accepted an endorsement from Wayne LaPierre that got his very long national political career started. He then paid them back with a vote to give gun manufacturers immunity from lawsuits. He is the only Dem candidate with a passing grade from the NRA at this point.
the weirdest thing about gender reveal parties is that they straight up didn't exist ten years ago and yet you have people treating them as

In [ ]:
#df = df.reset_index(drop=True)

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
#for text in df.text:
    #print(text)
df.text = df.text.apply(clean_text)
df['replies__text'] = df['replies__text'].apply(clean_text)

for i in df.text:
    column.append(i)
for i in df.replies__text:
    column.append(i)
#df.head()
#df.info
print("The number of comments and replies being considered:")
print(len(column))

The number of comments and replies being considered:
740


In [ ]:
cnt=0
words=[]
def ction(sentence):
    w=[]
    w=list(sentence.split(" "))
    #print(w)
    #print("HELLO")
    return(w)
    
for sentence in column:
    cnt=cnt+1
    if cnt>30:
        break
    w = ction(sentence)
    words.extend(w)
wors=[]
wors=words.copy()
print("The number of unique words and hence key-value pairs in the ditctionary in the first 30 comments: ")
#print (len(words) )
words = sorted(list(set(words)))
#print(words)
#return words
print (len(words) )# same as line 34 in this cell 
di={}

#print(type(di))
for i in set(words):
    di[i]=0

for word in wors:
    if '#' in word:
        di[word]+=5 ##this is to give high importance to hashtagged words
    di[word]+=1 ##
print(len(di))  
for key,value in di.items():
    print(str(key)+" ------------- "+str(value))
#print(len(column))        

The number of unique words and hence key-value pairs in the ditctionary in the first 30 comments: 
362
362
medical ------------- 2
banish ------------- 1
seanad ------------- 1
circumcision ------------- 1
ovarian ------------- 1
2 ------------- 1
let ------------- 2
things ------------- 1
doesnt ------------- 2
40 ------------- 1
nra ------------- 1
choose ------------- 1
less ------------- 1
know ------------- 1
family ------------- 1
little ------------- 1
yrs ------------- 1
theres ------------- 1
backed ------------- 1
retire ------------- 1
life ------------- 3
tell ------------- 1
epectancy ------------- 1
parker ------------- 1
screening ------------- 3
cervical ------------- 6
even ------------- 1
cant ------------- 1
increases ------------- 1
involved ------------- 1
girls ------------- 1
manufacturers ------------- 1
autism ------------- 1
rate ------------- 1
#cervicalscreening ------------- 6
yet ------------- 3
available ------------- 2
given ------------- 1
gun ---------

In [ ]:
di2=di.copy()
fqlist=[]
for (key, value) in di2.items() :
    #print(str(key)+" "+str(value))
    if (value<2):
        del di[key]
    else:
        fqlist.append(str(key))
for (key, value) in di.items() :
    print(str(key)+" ------"+str(value))
print(len(di))
#print(di)
#print(fqlist)
print("Original number of comments and replies "+str(len(column)))


medical ------2
let ------2
doesnt ------2
life ------3
screening ------3
cervical ------6
#cervicalscreening ------6
yet ------3
available ------2
take ------4
#hpvvaccine ------6
#savannahga ------6
mama ------4
smoking ------2
years ------3
boys ------5
study ------2
#cervicalcancer ------12
cancers ------3
uk ------3
like ------3
prevent ------2
diseases ------2
facts ------2
go ------4
dem ------2
dont ------5
10 ------2
ur ------6
include ------2
pap ------3
men ------2
risk ------3
thing ------2
vaccine ------5
say ------2
something ------2
give ------2
#vaccine ------6
isnt ------2
also ------2
crucial ------2
increasing ------2
evidence ------2
dey ------2
got ------2
hpv ------6
research ------2
vaccinations ------3
doctor ------2
passed ------2
cancer ------9
american ------2
#papsmear ------6
september ------2
goats ------2
choice ------2
change ------2
mum ------2
get ------3
lifestyle ------2
ago ------2
smear ------2
63
Original number of comments and replies 740


In [ ]:
print(len(column))
print("Original number of comments and replies :"+str(len(column)))
extracted=[]
sentwords=[]
rejected=[]
flag=0
for sentence in column:
    flag=0
    sentwords=list(sentence.split(" "))
    for word in sentwords:
        if word in fqlist:
            flag=1
            extracted.append(sentence)
            break
    if(flag==0):
        rejected.append(sentence)
        
print("Extracted number of comments and replies :"+str(len(extracted)))
print("Rejected number of comments and replies :"+str(len(rejected)))

740
Original number of comments and replies :740
Extracted number of comments and replies :606
Rejected number of comments and replies :134


In [ ]:
print("The extracted comments are:\n")
for e in extracted:
    print('"'+e+'\"\n')

The extracted comments are:

"dont agree subgroup study 013 unbalanced study 015 representative p14"

"sike feeling hurt daughter 9 months vaccinations left doctor healthy"

"monkey completely pointless tweet stick marveling seem smarter dont engage"

"sanders accepted endorsement wayne lapierre got long national political career started paid back vote give gun manufacturers immunity lawsuits dem candidate passing grade nra point"

"weirdest thing gender reveal parties straight didnt eist ten years ago yet people treating kind longstanding ubiquitous tradition"

"let little girl ever take #vaccine matter happens dont let take vaccine #hpvvaccine"

"yes using march 50 yrs ago tired retire"

"nursing tetbooks still racist shit stereotypes presented medical fact"

"looking another reason put cigarettes according american cancer society smoking increases risk cervical cancer also reduce risk getting hpv vaccine scheduling regular pap smear #papsmear #cervicalcancer #savannahga"

"circumcis

In [ ]:
print("The rejected comments are:\n")
for e in rejected:
    print('"'+e+'\"\n')

The rejected comments are:

"never skip test important #cervicalscreening"

"serves right voting"

"3rdplace regiontabora schooltabora girls secondary school studentsflora nyachiro nicodemus specioza judathadei kimaryo titlewater analysis disinfection"

"ils tentent tout les mecs"

"hillary think people still shooting helicopter"

"90 2024"

"big scary amazing news klaon one woman play madame ovary happening help going edinburgh summer please stay post constantly piece dear heart"

"review international literature suggests virus transmitted se relationships"

"eilis ohanlon judge leo record race seuality"

"good youyou dodged bullet"

"yall deserve whole freestyle absolutely insane flow lil dicky part 1"

"tweeted antisemitic tweets new account"

"recently watched episode series thats made start paying attention health please man tweet"

"jesus h christ goop shit utter garbage nonsense"

"great news #vaccineswork"

"high court confirmed suspension co cork vet returned wrong ashes owner